In [45]:
%%writefile process_funcs.py
import pandas as pd
import os
import glob
import numpy as np
import re
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import io

def process_excel_files(file_names, pattern):
    all_df = []

    for file_name in file_names:
        course_id = extract_pattern(str(file_name), pattern) 
        df = pd.read_excel(file_name, sheet_name='UBYS', header=1)
        
        # Add the "Course" column to the DataFrame
        df.insert(0, "Ders", course_id)
        all_df.append(df)

    return all_df

def preprocess_data(df):

    # Remove the last two columns
    df = df.iloc[:, :-2]

    # Drop rows where the second column is NaN
    df = df.dropna(subset=[df.columns[2]])

    # Select only the columns
    cols = ['Ders','Numara', 'Ad', 'Soyad', 'PÇ1', 'PÇ2', 'PÇ3', 'PÇ4', 'PÇ5', 'PÇ6', 'PÇ7', 'PÇ8','PÇ9', 'PÇ10', 'PÇ11']
    new_df = df[cols]

    new_df = new_df.assign(**new_df[['Numara']].astype(np.int64))
    
    # Return the preprocessed data
    return new_df

def extract_rows_by_numbers(data_frames, target_numbers):
    result_dfs = []

    # Iterate through each target number
    for target_number in target_numbers:
        extracted_rows = []

        # Iterate through each data frame
        for df in data_frames:
            # Filter rows based on the "Number" column
            filtered_rows = df[df['Numara'] == target_number]

            # Append the filtered rows to the list
            extracted_rows.append(filtered_rows)

        # Concatenate the filtered rows into a new data frame for the current target number
        result_df = pd.concat(extracted_rows, ignore_index=True)

        # Append the result data frame to the list
        result_dfs.append(result_df)

    return result_dfs

def create_pattern(s):
    """Given a string that consists of letters followed by digits, creates a pattern to use in regex.

    Args:
    given_string: A string that consists of only letters followed by digits e.g. "XYZ123".

    Returns:
    A string that represents a compiled regular expression pattern.
    """

    # Check if the given string is a valid string that consists of letters followed by digits.
    letters = sum(c.isalpha() for c in s)
    numbers = sum(c.isdigit() for c in s)

    # Create a regular expression pattern that matches the given string.
    pattern = '([A-Za-z]{' + str(letters) + '}\d{' + str(numbers) + '})'

    # Return the compiled regular expression pattern.
    return pattern

def extract_pattern(input_string, pattern):
#     pattern = r'([A-Za-z]{3}\d{3})'
    matches = re.findall(pattern, input_string)
    result = ' '.join(matches)
    return result

def delete_empty_dfs(dataframes, target_list):
    deleted_ids = []
    filtered_df_list = []

    for i, df in enumerate(dataframes):
        if df.empty:
            deleted_ids.append(target_list[i])
        else:
            filtered_df_list.append(df)
    return filtered_df_list, deleted_ids
    
def process_student_df(df1, df2):
    """
    Process and merge two DataFrames based on the 'Dersler' column, fill NaN values, and perform data conversions.

    Args:
        df1 (pd.DataFrame): The first DataFrame containing the 'Dersler' column.
        df2 (pd.DataFrame): The second DataFrame to be merged with df1. Ex: result_dfs[0]

    Returns:
        pd.DataFrame: The processed DataFrame with NaN values filled, data type conversions, and 'Ders' column dropped.
    """
    # Merge df1 and df2 using a left join based on the 'Dersler' column in df1 and the 'Ders' column in df2
    df_merged = df1[['Dersler']].merge(df2, how='left', left_on='Dersler', right_on='Ders')

    # Drop the 'Ders' column from df_merged, as it's no longer needed
    df_merged.drop('Ders', axis=1, inplace=True)

    # Define a list of columns to fill NaN values with the most common value
    columns_to_fill = ["Numara", "Ad", "Soyad"]

    # Iterate through the specified columns and replace NaN values with the most common value
    for column in columns_to_fill:
        most_common_value = df_merged[column].mode().iloc[0]  # Get the most common value
        df_merged[column].fillna(most_common_value, inplace=True)

    # Fill any remaining NaN values in df_merged with 0
    df_merged = df_merged.fillna(0)

    # Convert the 'Numara' column to integer data type
    df_merged['Numara'] = df_merged['Numara'].astype(np.int64)

    return df_merged

def calculate_weighted_means(df_values, df_weights):
    """
    Calculate the weighted means of columns in a DataFrame based on values and weights.

    Args:
        df_values (pd.DataFrame): DataFrame containing values for which weighted means are calculated.
        df_weights (pd.DataFrame): DataFrame containing weights for the values.

    Returns:
        pd.DataFrame: DataFrame containing the calculated weighted means.
    """
    # Calculate the weighted means for each column
    weighted_means = (df_values * df_weights).sum(axis=0) / df_weights.sum(axis=0)

    # Convert the Series of weighted means to a DataFrame
    weighted_means_df = weighted_means.to_frame().T

    # Reset the index if needed
    weighted_means_df.reset_index(drop=True, inplace=True)

    return weighted_means_df

def process_result_dfs_v5(df_students, df_pc_dersler):
    
    # Get the weights 
    df_weights = df_pc_dersler.iloc[:, 1:]
    
    # Create an empty list
    df_list = []
    
    # Iterate over the student dataframes
    for df_student in df_students:

        if df_student.empty:
          print('There is an empty DataFrame')
          continue

        # Create a temporary list to store student info
        temp_list1 = [df_student['Numara'][0],df_student['Ad'][0],df_student['Soyad'][0]]
        
    
        # Values from df_students
        df_values = df_student.iloc[:, 4:]
        
        # Calculate the weighted means for each student
        weighted_means = (df_values * df_weights).sum(axis=0) / df_weights.sum(axis=0)
        
        # Store the weighted means in a list
        temp_list2 = list(weighted_means.values)
        
        # Combine lists for each student
        temp_list = temp_list1 + temp_list2
        
        # List of lists (each sublist has one students info)
        df_list.append(temp_list)

    # Concatenate the DataFrames
    df = pd.DataFrame(df_list)
    
    df.columns = ['Numara', 'Ad', 'Soyad', 'PÇ1', 'PÇ2', 'PÇ3', 'PÇ4', 'PÇ5', 'PÇ6', 'PÇ7', 'PÇ8', 'PÇ9', 'PÇ10', 'PÇ11']
    
    # Replace NaN values with zero
    df = df.fillna(0)

    return df

def hide_names(df, columns_to_hide=None):
    if columns_to_hide is None:
        columns_to_hide = df.columns

    def hide_name(name):
        return name.lstrip()[0] + '*' * (5) if len(name) > 1 else name
    
    df_copy = df.copy()
    for column in columns_to_hide:
        if column in df_copy.columns:
            if df_copy[column].dtype == 'object':
                df_copy[column] = df_copy[column].apply(hide_name)
            elif df_copy[column].dtype == 'int64':
                df_copy[column] = df_copy[column].astype(str).apply(hide_name)
    return df_copy

def append_average_row(df):
    # Calculate the average for the specified columns
    average_row = {
        'Numara': 'ORTALAMA',
        'Ad': 'ORTALAMA',
        'Soyad': 'ORTALAMA'
    }

    # Filter out the zero values from the DataFrame before calculating the mean
    non_zero_df = df.replace(0, np.nan)

    for col in ['PÇ1', 'PÇ2', 'PÇ3', 'PÇ4', 'PÇ5', 'PÇ6', 'PÇ7', 'PÇ8', 'PÇ9', 'PÇ10', 'PÇ11']:
        average_row[col] = non_zero_df[col].mean()

    # Create a DataFrame from the average_row dictionary
    df_average = pd.DataFrame([average_row])
    
    # Replace NaN values with zero
    df = df.fillna(0)
    
    # Concatenate the DataFrames
    df = pd.concat([df, df_average], ignore_index=True)
    
    return df

Overwriting process_funcs.py


In [60]:
%%writefile main.py
import streamlit as st
import pandas as pd
import os
import glob
import numpy as np
import re
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from process_funcs import *

# Create a sidebar
st.sidebar.title("About")

# Write a description of the project
st.sidebar.write("This application calculates the rates at which graduates meet their program outcomes.")

st.sidebar.subheader('Instructions', divider='orange')

# Add a list of instructions
instructions = [
    "Enter a sample course ID",
    "Upload course evaulation reports.",
    "Upload the graduation list.",
    "Upload the course list file.",
    "Run the code and display results.",
    "Download the resulting Excel file."
]
for i, instruction in enumerate(instructions,1):
    st.sidebar.markdown(f"{i}. {instruction}")

header = st.container()
dataset = st.container()
results = st.container()

with header:
    st.title('Program Objectives Project')
    
    st.write("This application calculates the rates at which graduates meet their program outcomes.")
        
    # Create course id pattern
    st.markdown("**IMPORTANT NOTICE: Before proceeding please provide a sample course id pattern, e.g., CSS210. **")
    with st.form(key='my_form'):
        sample_id = st.text_input(label='Enter a sample course ID')
        submit_id_button = st.form_submit_button(label='Submit')
        
#     sample_id = st.text_input('Enter a sample course ID', 'XYZ101')
        course_id_pattern = create_pattern(sample_id)
    
with dataset:
    st.subheader('Step 1: Upload Course Evaulation Reports',divider='orange')
    
    # Sample report DataFrame
    ex_report = pd.read_excel('ex_report.xlsx')
    
    if st.checkbox("Display a Sample Course Evaulation Report", key = 'ex_report'):
        st.dataframe(ex_report)
   
    eval_files = st.file_uploader("Choose all the Excel files for course reports", accept_multiple_files=True, key='file_uploader1')
    
    st.write(f'{len(eval_files)} files are uploaded.')

    st.subheader('Step 2: Upload the Graduation List', divider = 'orange')
    
    # Sample Grad list DataFrame
    ex_grad_list = pd.read_excel('ex_grad_list.xlsx')

    if st.checkbox("Display a Sample Graduation List", key = 'ex_grads'):
        st.dataframe(ex_grad_list)
    
    # Create a file uploader
    grad_list_file = st.file_uploader("Choose an Excel file of graduation list", key='file_uploader2')
    
    # Read the uploaded file to a Pandas DataFrame
    if grad_list_file is not None:
        df_mezun_list = pd.read_excel(grad_list_file)
        disp_df = df_mezun_list.style.format(precision=0, thousands='')
        # Display the DataFrame
        st.write(disp_df)

    st.subheader('Step 3: Upload the Course List File', divider = 'orange')
    
    # Sample Weights DataFrame
    ex_weights = pd.read_excel('ex_weights.xlsx')
    
    if st.checkbox("Display a Sample Graduation List", key = 'ex_weights'):
        st.dataframe(ex_weights)
    
    # Create a file uploader
    course_list_file = st.file_uploader("Choose the Excel file of Course-Outcome relation", key='file_uploader3')

    # Read the uploaded file to a Pandas DataFrame
    if course_list_file is not None:
        df_pc_dersler = pd.read_excel(course_list_file)
        
        # Apply the extract_pattern function to each row in the 'Dersler' column
        df_pc_dersler['Dersler'] = df_pc_dersler['Dersler'].apply(lambda x: extract_pattern(x, course_id_pattern))

        # Display the DataFrame
        st.write(df_pc_dersler)
    
with results:
    if eval_files is not None and grad_list_file is not None and course_list_file is not None:
        if st.button("Get the Results", key = 'get_results'):
            with st.spinner("Loading the results"):
                # Get all excel files in the folder as list of dataframes
                all_df = process_excel_files(eval_files, course_id_pattern)

                # Preprocess these dataframes
                processed_df = [preprocess_data(df) for df in all_df]

                # Store the id's in a list
                mezun_list = list(df_mezun_list['Öğrenci No'])

                # Get result_dfs
                result_dfs = extract_rows_by_numbers(processed_df, mezun_list)

                # Remove the empty dataframes of students and get which ones are deleted
                result_dfs, deleted_ids = delete_empty_dfs(result_dfs, mezun_list)

                # Create a dataframe of deleted ones
                deleted_df = df_mezun_list[df_mezun_list['Öğrenci No'].isin(deleted_ids)]
                deleted_students = deleted_df.copy()
                deleted_students = deleted_df.style.format(precision=0, thousands='')
                
                st.subheader('Students not considered for evaluation', divider = "orange")
                st.write('These students are not included in the computations because all PC values are zero.')
                st.dataframe(deleted_students)

                st.subheader('Results', divider = 'orange')
                df_students = [process_student_df(df_pc_dersler, df2) for df2 in result_dfs]
                df = process_result_dfs_v5(df_students, df_pc_dersler)
                df = append_average_row(df)
                st.dataframe(df)

                st.subheader('Downloading the results as Excel file', divider = 'orange')
                # Create a download button
                def download_excel(df):
                    df.to_excel('dataframe.xlsx', index=False)
                    with open('dataframe.xlsx', 'rb') as f:
                        data = f.read()
                    st.download_button(
                        label="Download the results",
                        data=data,
                        key='download_excel',
                        file_name='dataframe.xlsx',
                        mime='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
                    )

                # Call the download button function
                download_excel(df)

                st.subheader('Data analysis', divider = 'orange')
                name = st.selectbox('Select a name:', df['Ad'])
                # Print the selected name
                st.write('You selected:', name)

Overwriting main.py
